In [ ]:
import pandas as pd
import numpy as np

from PIL import Image
import os
import sys
!pip install ipython-autotime

%load_ext autotime
%matplotlib inline


The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.43 s (started: 2021-01-22 18:15:43 +00:00)


1. Extract your dataset and split into train_x, train_y, test_x and test_y.
2. Execute the following cells



---


## Hybrid Social Group Optimization 


---




In [ ]:
N = 5 # Number of persons in population
D = len(train_x.columns) # Number of features in dataset
g = 10 # Number of generations
c = 0.6 # Self Introspection factor
r0 = 1
r1 = 0.4
r2 = 0.6
print(r1, r2)

0.4 0.6
time: 3.13 ms (started: 2021-01-22 18:19:07 +00:00)


**Population Initialization**

In [ ]:
population = np.random.choice([0,1,2,3,4,5,6,7,8,9], (N,D), p=[0.16, 0.16, 0.16, 0.16, 0.16, 0.04, 0.04, 0.04, 0.04, 0.04]) #Determines no. of features selected by probablity

time: 5.05 ms (started: 2021-01-22 18:19:07 +00:00)


In [ ]:
population = population.astype(float)
print(population.shape)
population

(5, 10000)


array([[1., 4., 5., ..., 4., 7., 2.],
       [2., 7., 2., ..., 3., 2., 3.],
       [3., 2., 4., ..., 1., 4., 4.],
       [3., 7., 3., ..., 0., 4., 7.],
       [2., 3., 1., ..., 0., 3., 1.]])

time: 5.69 ms (started: 2021-01-22 18:19:07 +00:00)


In [ ]:
fitness = np.zeros(N)
test_x.shape

(294, 10000)

time: 2.92 ms (started: 2021-01-22 18:19:07 +00:00)


In [ ]:
def fitter_trait(X_old, X_new):
  if X_new > X_old:
    return X_old
  else:
    return X_new
  
  

time: 2.58 ms (started: 2021-01-22 18:19:07 +00:00)







**Fitness Function**

In [ ]:
global classifier
classifier = Svc #Change classifier here
select = train_x.columns
selectno = len(train_x.columns)
classifier.fit(train_x, train_y)
select_acc = classifier.score(test_x, test_y) 
def fitness_function(pop):  #Fitness Function
  
  for i in range(N):
    new_train_x = train_x
    
    new_test_x = test_x
    
    global select
    global selectno
    global select_acc
    
    new_train_x = new_train_x.drop(train_x.columns[pop[i] < 4], axis = 1)
        
    new_test_x = new_test_x.drop(test_x.columns[pop[i] < 4], axis = 1)
        
    classifier.fit(new_train_x, train_y)  
    fitness[i] = classifier.score(new_test_x, test_y) 
    if (fitness[i] > select_acc):
      select = new_train_x.columns
      # print(select.shape)
      selectno = new_train_x.shape[1]
      select_acc = fitness[i]
    elif fitness[i] == select_acc and new_train_x.shape[1] < selectno:
      select = new_train_x.columns
      selectno = new_train_x.shape[1]
      
    print("\nPerson "+ str(i+1))
    
    print("No. of Features Used = "+ str(new_train_x.shape[1])+ "/"+str(D)+"\nFitness = " + str(fitness[i]))
    print("Feature Used = ", end = " ") 
    
    #print(new_train_x.columns)

    print(new_train_x.shape[1])
    

time: 5.47 s (started: 2021-01-22 18:19:07 +00:00)


In [ ]:
# Initializing Fitness values of population
# fitness_function(population) 
# selectno

time: 785 µs (started: 2021-01-22 18:19:13 +00:00)


**Gbest : Fittest person in population**

In [ ]:
###Determining GBest
gbest = 0
gbest_i = 0

time: 874 µs (started: 2021-01-22 18:19:13 +00:00)


In [ ]:
def find_gbest():
  gbest = max(fitness)#This can be any function
  gbest_i = fitness.argmax()
  print("Best fitness value for the generation = "+str(gbest) + " Person " + str(gbest_i+1)+"\n")
find_gbest() 
#we chose maximum fitness value to be better for simplicity
def cal_fitness(person):
  new_train_x = train_x
  
  new_test_x = test_x

  new_train_x = new_train_x.drop(train_x.columns[person < 4], axis = 1)
      
  new_test_x = new_test_x.drop(test_x.columns[person < 4], axis = 1)
      
  classifier.fit(new_train_x, train_y)  
  return classifier.score(new_test_x, test_y)

cal_fitness(population[0])

Best fitness value for the generation = 0.0 Person 1



0.9863945578231292

time: 1.95 s (started: 2021-01-22 18:19:13 +00:00)


In [ ]:
# new_train_x = train_x
  
# new_test_x = test_x

# new_train_x = new_train_x.drop(train_x.columns[person < 4], axis = 1)
      
# new_test_x = new_test_x.drop(test_x.columns[person < 4], axis = 1)

time: 913 µs (started: 2021-01-22 18:19:15 +00:00)


In [ ]:
per1 = np.zeros((1,10000))
print(per1.shape)
per1[0][5] = 8
per1[0][89] = 7
per1[0][45] = 6
cal_fitness(per1[0])

(1, 10000)


0.935374149659864

time: 41.7 ms (started: 2021-01-22 18:19:15 +00:00)




---

**Mutation Phase**

In [ ]:
def mutate():
  gworst_i = fitness.argmin()
  gworst = min(fitness)
  mut = np.random.randint(0,2,size=(1,D))[0]
  print("Mutating the Generation's Worst....Person "+ str(gworst_i+1))
  for i in range(D):
    if mut[i] > 0:
      mut[i] = population[gbest_i][i]
    else:
      mut[i] = population[gworst_i][i]
  if cal_fitness(mut) > gworst:
    population[gworst_i] = mut
    print("Person "+str(gworst_i)+" mutated")
  else:
    print("No Mutations in this generation")

time: 5.35 ms (started: 2021-01-22 18:19:15 +00:00)


In [ ]:
mut = np.random.randint(0,2,size=(1,D))[0]
mut

array([1, 0, 0, ..., 0, 0, 0])

time: 2.99 ms (started: 2021-01-22 18:19:15 +00:00)


In [ ]:
div = pd.DataFrame(np.random.randint(0,2,size=(1,D))[0])
# div.iloc[:,div > 0] = population[2][div>0]
# div

time: 1.79 ms (started: 2021-01-22 18:19:15 +00:00)




---

**Improving Phase**

In [ ]:
## Improving Phase
# i = 1
def improve():
  print("Improving.......")
  for i in range(N):
    Xnew = population[i]
    print('Persona '+ str(i+1))
    for j in range(D):
      Xnew[j] = c * population[i][j] + r0 * (population[gbest_i][j] - population[i][j])
    try:
      if cal_fitness(Xnew) > fitness[i]:
        population[i] = Xnew
    except:
      print("Oops!", sys.exc_info()[0], "occurred.")
      print("Next entry.")
      


time: 4.73 ms (started: 2021-01-22 18:19:15 +00:00)




---

**Acquiring Phase**

In [ ]:
## Acquiring Phase
def acquire():
  random_person = np.random.randint(low=0, high=N)
  for i in range(N):
    if random_person == i:
      random_person = np.random.randint(low=0, high=N)
      i = i - 1
      continue
    X_new = population[i]
    if fitness[random_person] > fitness[i]:
      for j in range(D):
        X_new[j] = population[i][j] + r1*(population[random_person][j]-population[i][j]) + r2*(population[gbest_i][j]-population[i][j])
      if cal_fitness(X_new) > fitness[i]:
        population[i] = X_new
    else:
      for j in range(D):
        X_new[j] = population[i][j] + r1*(population[i][j]-population[random_person][j]) + r2*(population[gbest_i][j]-population[i][j])
      if cal_fitness(X_new) > fitness[i]:
        population[i] = X_new
      


time: 8.58 ms (started: 2021-01-22 18:19:15 +00:00)


In [ ]:
#Run
try:
  for k in range(g):
    print("Generation "+ str(k+1) + "\n---------------")
    fitness_function(population)
    find_gbest()
    mutate()
    improve()
    acquire() 
except:
  print()
  print("........................")
  print("Optimal Solution Reached")
  print("........................")

Generation 1
---------------

Person 1
No. of Features Used = 3582/10000
Fitness = 0.9863945578231292
Feature Used =  3582

Person 2
No. of Features Used = 3608/10000
Fitness = 0.9897959183673469
Feature Used =  3608

Person 3
No. of Features Used = 3614/10000
Fitness = 0.9897959183673469
Feature Used =  3614

Person 4
No. of Features Used = 3570/10000
Fitness = 0.9897959183673469
Feature Used =  3570

Person 5
No. of Features Used = 3581/10000
Fitness = 0.9897959183673469
Feature Used =  3581
Best fitness value for the generation = 0.9897959183673469 Person 2

Mutating the Generation's Worst....Person 1
No Mutations in this generation
Improving.......
Persona 1
Persona 2
Persona 3
Persona 4
Persona 5
Generation 2
---------------

Person 1
No. of Features Used = 830/10000
Fitness = 0.9897959183673469
Feature Used =  830

Person 2
No. of Features Used = 660/10000
Fitness = 0.9863945578231292
Feature Used =  660

Person 3
No. of Features Used = 647/10000
Fitness = 0.9829931972789115
Feat

In [ ]:
select.shape

(514,)

time: 2.19 ms (started: 2021-01-22 18:19:32 +00:00)
